In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm_notebook
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Observe the train data

In [ ]:
trainData = pd.read_csv("../input/conways-reverse-game-of-life-2020/train.csv", index_col=0)
trainData.head()

In [ ]:
trainData.duplicated().sum()

In [ ]:
print("Number before dropping : %d " % trainData.shape[0])
trainData = trainData.drop(index=trainData[trainData.duplicated()].index)
print("Number after dropping : %d " % trainData.shape[0])

# Valid and Generate More Boards

In [ ]:
N_ROWS = 25
N_COLS = 25
def getFlowBoards(index):
    start = trainData.iloc[index][1:626].to_numpy()
    startBoard = start.reshape(25, 25)

    delta = 5
    boards = {0: startBoard}
    tmp = np.array(startBoard, copy=True)
    for i in range(delta):
        for row in range(N_ROWS):
            for col in range(N_COLS):
                current = boards[i][row][col]
                arround = boards[i].take(range(row-1,row+2),mode='wrap', axis=0).take(range(col-1,col+2),mode='wrap',axis=1)
                arroundSum = arround.sum()
                arroundSum = arroundSum - current
                if current == 0 and arroundSum == 3:
                    tmp[row][col] = 1
                elif current == 1:
                    if arroundSum > 3 or arroundSum < 2:
                        tmp[row][col] = 0
                    else:
                        tmp[row][col] = 1
        boards[i + 1] = np.array(tmp, copy=True)
    return boards;


In [ ]:
allBoards = {}
for i in tqdm_notebook(range(trainData.shape[0])):
    gameBoards = getFlowBoards(i)
    delta = trainData.iloc[i][0]
    stop = trainData.iloc[i][626:].to_numpy()
    stopBoard = stop.reshape(25, 25)
    if (np.array_equal(gameBoards[delta], stopBoard)):
        allBoards[i] = gameBoards
    
print("{} boards are validated".format(len(allBoards)))


In [ ]:
delta1TrainSource = []
delta1TrainTarget = []
delta2TrainSource = []
delta2TrainTarget = []
delta3TrainSource = []
delta3TrainTarget = []
delta4TrainSource = []
delta4TrainTarget = []
delta5TrainSource = []
delta5TrainTarget = []
allBoards


for b in allBoards.values():
    for i in range(len(b) - 1):
        delta1TrainSource.append(b[i + 1])
        delta1TrainTarget.append(b[i])
      
    for i in range(len(b) - 2):
        delta2TrainSource.append(b[i + 2])
        delta2TrainTarget.append(b[i])

    for i in range(len(b) - 3):
        delta3TrainSource.append(b[i + 3])
        delta3TrainTarget.append(b[i])
    
    for i in range(len(b) - 4):
        delta4TrainSource.append(b[i + 4])
        delta4TrainTarget.append(b[i])
        
    for i in range(len(b) - 5):
        delta5TrainSource.append(b[i + 5])
        delta5TrainTarget.append(b[i])
            
print("number of delta 1 data = {0}".format(len(delta1TrainSource)))
print("number of delta 2 data = {0}".format(len(delta2TrainSource)))
print("number of delta 3 data = {0}".format(len(delta3TrainSource)))
print("number of delta 4 data = {0}".format(len(delta4TrainSource)))
print("number of delta 5 data = {0}".format(len(delta5TrainSource)))

# Save to CSV

In [ ]:
trainSource = np.array([delta1TrainSource, delta2TrainSource, delta3TrainSource, delta4TrainSource, delta5TrainSource])
trainTarget = np.array([delta1TrainTarget, delta2TrainTarget, delta3TrainTarget, delta4TrainTarget, delta5TrainTarget])

In [ ]:
for i in tqdm_notebook(range(5)):
    pd.DataFrame(data=np.array(trainSource[i]).reshape(-1, 625), columns=['start_' + str(_) for _ in range(0, 625)]).to_csv("train_delta{}_source.csv".format(i+1))
    pd.DataFrame(data=np.array(trainTarget[i]).reshape(-1, 625), columns=['start_' + str(_) for _ in range(0, 625)]).to_csv("train_delta{}_target.csv".format(i+1))